In [9]:
import single_head
import multi_head
import copy
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy.ndimage.morphology import binary_fill_holes
import sys
import pickle
np.set_printoptions(threshold=sys.maxsize)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# load heads
all_heads=[single_head.SingleHead.load_from_pickle(1,i) for i in range (1,16)]

In [14]:
# make head from first to last
def make_mhead(all_heads, first, last):
    all_heads[first].reset_positions()
    all_heads[first].reset_colors()
    all_heads[first+1].reset_positions()
    all_heads[first+1].reset_colors()   
    heads = multi_head.MultiHead.joined_heads(all_heads[first], all_heads[first+1])
    heads.links.append(multi_head.Link(left=all_heads[first+1].frame_id,right=all_heads[first].frame_id))
    for i in range(first+2,last+1):
        heads.links.append(multi_head.Link(left=all_heads[i].frame_id,right=all_heads[i-1].frame_id))
        all_heads[i].reset_positions()
        all_heads[i].reset_colors()
        heads.append(all_heads[i])        
        if i==last:
             heads.links.append(multi_head.Link(left=all_heads[first].frame_id,right=all_heads[i].frame_id))
    print ("completed")
    return heads
mhead = make_mhead(all_heads, 0,14)
mhead.calc_keypoints()

completed


In [16]:
def calc_connections(mhead):
    for link in mhead.links:
        link.reset()
        mhead.ransac_from_link(link)        
calc_connections(mhead)

/Users/caseliang/Repositories/AV_CW/SIFT.py:91: RuntimeWarning: divide by zero encountered in double_scalars
  err = np.sqrt(np.var(dist) / (np.sum(inliers) - min_num_inliers))
/Users/caseliang/Repositories/AV_CW/SIFT.py:91: RuntimeWarning: invalid value encountered in sqrt
  err = np.sqrt(np.var(dist) / (np.sum(inliers) - min_num_inliers))


In [ ]:
only_first_n=15
mhead.reset_all_head_positions()
link_index,err=mhead.get_next_unpositioned_link()
positioned_head_count=0
joined_heads=set() 
while(not link_index is None) and (positioned_head_count<only_first_n or only_first_n==-1):
    mhead.links[link_index].print_short()
    joined_heads.add(mhead.links[link_index].left)
    joined_heads.add(mhead.links[link_index].right)
    foo= mhead.transform_from_link(mhead.links[link_index])
    link_index,err=mhead.get_next_unpositioned_link()
    positioned_head_count=max(positioned_head_count+1,2)
joined_heads=sorted(joined_heads)
print(joined_heads)
no_heads= len(joined_heads)
joined_heads=[str(h) for h in joined_heads]
name= f"Seq_{mhead.heads[0].sequence_id}_frames_{'_'.join(joined_heads)}"
mhead.create_spheres(min(2/max(no_heads,0.2),1),name=name)

In [ ]:
mhead.create_mesh('mesh')

In [85]:
for i, link in enumerate(mhead.links):
    print (i)
    link.print_short()

0
2-1, Count=78, Err=0.0029
1
3-2, Count=86, Err=0.0037
2
4-3, Count=53, Err=0.0053
3
5-4, Count=80, Err=0.0028
4
6-5, Count=67, Err=0.0051
5
7-6, Count=52, Err=0.0055
6
8-7, Count=39, Err=0.0030
7
9-8, Count=12, Err=0.0130
8
10-9, Count=5, Err=nan
9
11-10, Count=19, Err=0.0028
10
12-11, Count=27, Err=0.0076
11
13-12, Count=39, Err=0.0030
12
14-13, Count=34, Err=0.0068
13
15-14, Count=42, Err=0.0076
14
1-15, Count=92, Err=0.0028


In [100]:
mhead.reset_all_head_positions()
mhead.transform_from_link(mhead.links[13])
mhead.create_spheres()

In [32]:
mhead.create_mesh("mesh_head")


completed


In [ ]:
# bad 11 12
# 12-11, Count=53, Err=0.0077

second

In [ ]:
#using edge diameter 20, 2 nearest neighbors, 0.9 threshold
#using edge diameter 20, 1 nearest neighbors, 0.4 threshold